In [ ]:
# %load_ext autoreload
# %autoreload 2
import sys
sys.path.append("/home/qrr2/project/")
sys.path.append("/home/qrr2/project/QOGS/")
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import os
# os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_a"sync" # this seems to be highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown
# note that GradientTape needs several times the memory needed to compute the fidelity of a single circuit
import tensorflow as tf
import numpy as np
import qutip as qt
import h5py
import pandas as pd
from QOGS.optimizer.tf_adam_optimizer import AdamOptimizer
from QOGS.gate_sets import GRAPE
import matplotlib.pyplot as plt

### Definitions

In [ ]:
dim_qubit = 2                                     
dim_resonator = 6
fock = 3 # Target fock interger
DAC_time_resolution = 4 # Pulse length/number of time slices
pulse_length = 3600 # Pulse length in ns
g = 0.00028 * np.pi * 2 # In GHz
Id = qt.tensor(qt.qeye(dim_qubit), qt.qeye(dim_resonator))
q = qt.tensor(qt.destroy(dim_qubit), qt.qeye(dim_resonator)) #qubit
r = qt.tensor(qt.qeye(dim_qubit), qt.destroy(dim_resonator)) #phonon
qd = q.dag() #qubit
rd = r.dag() #phonon

H0 = g * (rd * q + r * qd)
Hcs = [q + qd, 1j*(q - qd)]

init_states = [qt.tensor(qt.basis(dim_qubit, 0), qt.basis(dim_resonator, 0))]
final_states = [qt.tensor(qt.basis(dim_qubit, 0), qt.basis(dim_resonator, fock))]

### Optimization

In [ ]:
name = "GRAPE_control"

synth_params = {
"N_blocks": int(pulse_length/DAC_time_resolution), 
"N_multistart" : 45, # Number of parallel circuit optimizations 
"epochs" : 5000, # Number of epochs before termination
"epoch_size" : 5, # Number of adam steps per epoch
"learning_rate" : tf.keras.optimizers.schedules.PiecewiseConstantDecay([50, 250], [0.2, 0.2, 0.1]), # Adam learning rate
"term_fid" : 0.995, # Target fidelitiy
"dfid_stop" : -1, # Stop if dfid is smaller than this number
"initial_states" : init_states, 
"target_states" : final_states, 
"name" : name, 
"coherent" : True,
"filename" : None, 
}

# Initialize the gateset 
gate_set_params = {
    "H_static" : H0,
    "H_control" : Hcs,
    "DAC_delta_t" : DAC_time_resolution,
    "inplace" : False, 
    "scale" : 2, 
    "bandwidth" : 0.1/2, 
    "gatesynthargs": synth_params
}

GRAPE_gate_set = GRAPE(**gate_set_params)

opt = AdamOptimizer(GRAPE_gate_set)
GRAPE_gate_set.best_fidelity()
opt.optimize()#logdir="logs")

### Pulse

In [ ]:
# Define the path to the pulse sequence file
path_to_file = "/home/qrr2/project/QOGS/examples/HyQu - Rumman/"
name = path_to_file + "GRAPE_control.h5"

with h5py.File(name, "r") as f:
    key = list(f.keys())[-1]
    pulse_obj = f[key]
    fids = pulse_obj["fidelities"][:]
    pulse_idx = np.argmax(np.amax(fids, axis=0))
    controls = []
    # Extract the I and Q control pulses for the pulse with the highest fidelity
    for k in range(1):
        controls.append(pulse_obj["I" + str(k)][-1, pulse_idx, :])
        controls.append(pulse_obj["Q" + str(k)][-1, pulse_idx, :])
    # Extract the DAC delta time from the pulse sequence attributes
    dt = pulse_obj.attrs["DAC_delta_t"]

pulse_len = controls[0].shape[0] * dt
times = np.arange(pulse_len, step=dt, dtype=float)
print(np.max(fids))
print(key)

# Plot I and Q
fig, ax = plt.subplots(figsize=(12, 4))
ax.set_facecolor("whitesmoke")
plt.plot(times, 1000*controls[0], label="Real", color="#440154FF")  # Teal
plt.plot(times, 1000*controls[1], label="Imaginary", color="#2C728EFF")  # Magenta
plt.xlabel("Time (ns)")
plt.ylabel("Control Amplitude (MHz)")
plt.title("GRAPE Pulse Controls")
plt.legend()
plt.show()

### Calculate Fidelity

In [ ]:
init_state = qt.tensor(qt.basis(dim_qubit, 0), qt.basis(dim_resonator, 0))
fin_state = qt.tensor(qt.basis(dim_qubit, 0), qt.basis(dim_resonator, fock))

fidelities = []

H0 = g * (rd * q + r * qd)
Hcs = [q + qd, 1j*(q - qd)]
H_tot = [H0] + [[Hc, ctrl] for Hc, ctrl in zip(Hcs, controls)]

# Simulate the evolution of the system
result = qt.mesolve(H_tot, init_state, times, [], [])

# Calculate the fidelity
final_state = result.states[-1]
fidelity = qt.fidelity(final_state, fin_state)

print(f"State {1} Fidelity: {fidelity}")

### Plot Density Matrix

In [ ]:
final_state = result.states[-1]
# Compute the density matrix of the final state
density_matrix = final_state * final_state.dag()

# Convert the density matrix to a NumPy array
density_matrix_np = density_matrix.full()

# Visualize the density matrix
fig, ax = qt.hinton(density_matrix)
plt.title('Density Matrix')
plt.show()

### Plot Average Population Evolution

In [ ]:
# Number operator for the resonator
n_r = qt.tensor(qt.qeye(dim_qubit), qt.num(dim_resonator))

# List of initial states for which to plot the population evolution
initial_states_to_plot = [qt.tensor(qt.basis(dim_qubit, 0), qt.basis(dim_resonator, 0))]

for init_state in initial_states_to_plot:
    H0 = g * (rd * q + r * qd)
    Hcs = [q + qd, 1j*(q - qd)]
    H_tot = [H0] + [[Hc, ctrl] for Hc, ctrl in zip(Hcs, controls)]

    # Simulate the evolution
    result = qt.mesolve(H_tot, init_state, times, [], e_ops=[n_r])
    
    # Plotting
    plt.figure(figsize=(10, 6))

    # Plot the expectation values for the resonator
    plt.plot(times, result.expect[0], lw=2, label='Resonator')

    plt.xlabel('Time (ns)', fontsize=12)
    plt.ylabel('Average Population', fontsize=12)
    plt.title(f'Average Population in HBAR Over Time', fontsize=14)
    plt.legend(fontsize=10)
    plt.tight_layout()
    plt.show()

### Save CSV

In [ ]:
def h5_to_expanded_csv(h5_filepath, csv_filepath):
    with h5py.File(h5_filepath, 'r') as f:
        key = list(f.keys())[-1]
        pulse_obj = f[key]
        fids = pulse_obj['fidelities'][:]
        pulse_idx = np.argmax(np.amax(fids, axis=0))
        I_pulse = pulse_obj['I0'][-1, pulse_idx, :] * 1000
        Q_pulse = pulse_obj['Q0'][-1, pulse_idx, :] * 1000
        
        expanded_rows = []
        for i, q in zip(I_pulse, Q_pulse):
            for _ in range(4):
                expanded_rows.append([i, q])
        
        expanded_df = pd.DataFrame(expanded_rows)
        expanded_df.insert(0, 'Time', range(len(expanded_df)))
        expanded_df.to_csv(csv_filepath, index=False, header=False)
        print(f"Modified CSV has been saved to {csv_filepath}")
        
h5_filepath = '/home/qrr2/project/QOGS/examples/HyQu - Rumman/GRAPE_control.h5'
csv_filepath = '/home/qrr2/project/QOGS/examples/HyQu - Rumman/GRAPE_control.csv'
h5_to_expanded_csv(h5_filepath, csv_filepath)

# To Prepare Cats

In [ ]:
dim_qubit = 2                                     
dim_resonator = 15
alpha = 3
DAC_time_resolution = 4
g = 0.00028 * np.pi * 2
Id = qt.tensor(qt.qeye(dim_qubit), qt.qeye(dim_resonator))
q = qt.tensor(qt.destroy(dim_qubit), qt.qeye(dim_resonator)) 
r = qt.tensor(qt.qeye(dim_qubit), qt.destroy(dim_resonator)) 
qd = q.dag() # Qubit
rd = r.dag() # Phonon

H0 = g * (rd * q + r * qd)
Hcs = [q + qd, 1j*(q - qd)]

init_states = [qt.tensor(qt.basis(dim_qubit, 0), qt.basis(dim_resonator, 0))]

# Create even and odd cat states for the resonator
cat_plus = (qt.coherent(dim_resonator, alpha) + qt.coherent(dim_resonator, -alpha)).unit()
cat_minus = (qt.coherent(dim_resonator, alpha) - qt.coherent(dim_resonator, -alpha)).unit()

# Define a four-legged cat state
cat_four_legged = (qt.coherent(dim_resonator, alpha) + qt.coherent(dim_resonator, -alpha) +
                   qt.coherent(dim_resonator, 1j * alpha) + qt.coherent(dim_resonator, -1j * alpha)).unit()

final_resonator_state = cat_four_legged
final_states = [qt.tensor(qt.basis(dim_qubit, 0), final_resonator_state)]

# To Prepare GKP

In [ ]:
dim_qubit = 2                                     
dim_resonator = 30
DAC_time_resolution = 4
g = 0.00028 * np.pi * 2
Id = qt.tensor(qt.qeye(dim_qubit), qt.qeye(dim_resonator))
q = qt.tensor(qt.destroy(dim_qubit), qt.qeye(dim_resonator)) 
r = qt.tensor(qt.qeye(dim_qubit), qt.destroy(dim_resonator)) 
qd = q.dag() # Qubit
rd = r.dag() # Phonon

H0 = g * (rd * q + r * qd)
Hcs = [q + qd, 1j*(q - qd)]

init_states = [qt.tensor(qt.basis(dim_qubit, 0), qt.basis(dim_resonator, 0))]

# Function to create a finite-energy GKP state
def finite_energy_gkp_state(N, d, alpha=np.sqrt(np.pi), epsilon=0.1):
    psi = basis(N, 0)  # initial state
    for m in range(-d, d + 1):
        for n in range(-d, d + 1):
            displacement_operator = displace(N, m * alpha) 
                                  * displace(N, 1j * n * alpha)
            state = displacement_operator * basis(N, 0)
            fock_damping = (1 / np.sqrt(np.cosh(epsilon))) 
                         * (-epsilon * num(N)).expm()
            state = fock_damping * state
            psi += state
    return psi.unit()

# Define parameters
d = 3  # Grid size

alpha = np.sqrt(np.pi)  # Lattice spacing in phase space
epsilon = 0.1  # Finite energy parameter

# Create the finite-energy GKP state
psi_gkp = finite_energy_gkp_state(dim_resonator, d, alpha, epsilon)
final_states = [qt.tensor(qt.basis(dim_qubit, 0), psi_gkp)]